## Setup

In [0]:
!pip install tensorflow-gpu>=2.0b1 gpustat -qU

In [0]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

import tensorflow

import tensorflow.compat.v2 as tf
import tensorflow.keras.layers as layers
import tensorflow_datasets as tfds

W0804 07:07:08.037279 140432171403136 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



# Transfer Learning with Keras

In this notebook, we are going to be doing image classification on the [Kaggle "Cats vs Dogs" dataset](https://www.kaggle.com/c/dogs-vs-cats).

However, we are not going to be training the model from scratch. Instead, we will be leveraging **transfer learning** to reduce the amount of time needed to train a very high accuracy model on a relatively small dataset.

This notebook demonstrates a typical example of transfer learning where we take a model that has been trained on ImageNet, and fine-tune it for our own task.

In [0]:
!gpustat

f3644b6b817d         Sun Aug  4 07:07:08 2019  410.79
[0] Tesla T4         | 67'C,   0 % |     0 / 15079 MB |


## Construct Input Pipeline

This section demonstrates how to build a high-performance input pipeline for feeding in training and evaluation data into the model.

### Load Dataset using TensorFlow Datasets

"Cats vs Dogs" is not a standard research dataset, and is not found in `tf.keras.datasets` like the more common MNIST or CIFAR10. Fortunately, **TensorFlow Datasets** not only contains this dataset, but also many others. 

When we load datasets with TensorFlow Datasets, we get [`tf.data.Dataset`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/data/Dataset) objects. This means we automatically take advantage of high performance `tf.data` primitives to create highly-optimized, efficient input pipelines that result in better utilization of our hardware accelerators, such as GPUs or TPUs that might otherwise be constantly "waiting" for data from the CPU. This is important because hardware accelerators for deep learning are so fast, the speed at which we can feed images into the accelerator is usually a significant bottleneck. 

Using `tf.data` is a best practice (especially for reading large datasets). You can read more about using tf.data at the following links:

* https://www.tensorflow.org/beta/guide/data
* https://www.tensorflow.org/guide/performance/datasets

In [0]:
# define train-validation-test split
splits = tfds.Split.TRAIN.subsplit(weighted=(8, 1, 1))

# load dataset with corresponding split
(raw_train, raw_validation, raw_test), info = tfds.load("cats_vs_dogs",
                                                        split=list(splits),
                                                        with_info=True,
                                                        as_supervised=True)

num_examples = info.splits["train"].num_examples
num_train = int(num_examples * 0.8)
num_val = int(num_examples * 0.1)
num_test = int(num_examples * 0.1)

print("Split:", num_train, num_val, num_test)

Split: 18609 2326 2326


### Build `tf.data` Pipeline

In [0]:
BATCH_SIZE = 120
IMG_SIZE = (256, 256)

@tf.function
def format_example(image, label):
    """
    This function will run as part of a tf.data pipeline.
    It is reponsible for resizing and normalizing the input images.
    """
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, IMG_SIZE)
    image = (image/127.5) - 1
    return image, label

**Input Pipeline for Train Images**

In [0]:
train = raw_train.shuffle(4096)
train = train.repeat(count=-1)
train = train.map(format_example, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train = train.batch(BATCH_SIZE)
train = train.prefetch(tf.data.experimental.AUTOTUNE)

**Input Pipeline for Validation and Test Images**

We remove some unnecessary steps (like shuffling) and increase the batch size for faster inferencing.

In [0]:
val = raw_validation.repeat(count=-1)
val = val.map(format_example, num_parallel_calls=tf.data.experimental.AUTOTUNE)
val = val.batch(BATCH_SIZE*2)
val = val.prefetch(tf.data.experimental.AUTOTUNE)

test = raw_test.repeat(count=-1)
test = test.map(format_example, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test = test.batch(BATCH_SIZE*2)
test = test.prefetch(tf.data.experimental.AUTOTUNE)

## Construct the Model

In [0]:
# you could also use other models like the following ResNet 50:
# from tensorflow.keras.applications.resnet50 import ResNet50

from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

In [0]:
input_layer = layers.Input(shape=(IMG_SIZE[0],IMG_SIZE[1],3,))

base = MobileNetV2(input_tensor=input_layer, include_top=False, weights='imagenet')

base.trainable = False

x = base.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
preds = layers.Dense(2, activation="softmax")(x)

model = tf.keras.models.Model(inputs=input_layer, outputs=preds)

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '
W0804 07:07:11.347321 140432171403136 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
opt = tf.keras.optimizers.Adam(lr=0.001)

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=opt,
              metrics=["accuracy"])

## Train Model

In [0]:
model.fit(train, validation_data=val,
          validation_steps=num_val//BATCH_SIZE*2,
          steps_per_epoch=num_train//BATCH_SIZE,
          epochs=5)

Epoch 1/5
155/155 [==============================] - 88s 567ms/step - loss: 0.1902 - acc: 0.9199 - val_loss: 0.1069 - val_acc: 0.9659
Epoch 2/5
155/155 [==============================] - 78s 501ms/step - loss: 0.0910 - acc: 0.9656 - val_loss: 0.1059 - val_acc: 0.9625
Epoch 3/5
155/155 [==============================] - 77s 499ms/step - loss: 0.0834 - acc: 0.9683 - val_loss: 0.0672 - val_acc: 0.9750
Epoch 4/5
155/155 [==============================] - 80s 516ms/step - loss: 0.0742 - acc: 0.9723 - val_loss: 0.0720 - val_acc: 0.9750
Epoch 5/5
155/155 [==============================] - 81s 522ms/step - loss: 0.0653 - acc: 0.9748 - val_loss: 0.0974 - val_acc: 0.9675
